In [ ]:
# Created by:  Sayed Ayaan
# Date : 19/3/2025

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/scholar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/scholar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# !pip install spacy

In [ ]:
# !pip install kagglehub

In [ ]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")
print("Path to dataset files:", path)

Path to dataset files: /home/scholar/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1


In [ ]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("sid321axn/amazon-alexa-reviews")
print("Path to dataset files:", path)

Path to dataset files: /home/scholar/.cache/kagglehub/datasets/sid321axn/amazon-alexa-reviews/versions/3


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score,recall_score,f1_score

In [ ]:
!ls /home/scholar/.cache/kagglehub/datasets/sid321axn/amazon-alexa-reviews/versions/3

amazon_alexa.tsv


In [ ]:
df_imdb = pd.read_csv("/home/scholar/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1/IMDB Dataset.csv")
df_amazon = pd.read_csv("/home/scholar/.cache/kagglehub/datasets/sid321axn/amazon-alexa-reviews/versions/3/amazon_alexa.tsv",sep='\t')

In [ ]:
df_imdb['sentiment'] = [1 if sentiment == 'positive' else 0 for sentiment in df_imdb['sentiment']]

In [ ]:
lemmatizer = WordNetLemmatizer()
STOPWORDS = set(stopwords.words('english'))
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])


def clean(df,col):
    df[col] = df[col].apply(lambda x:x.lower())
    df[col] = df[col].apply(lambda text: remove_stopwords(text))
    df[col] = df[col].apply(lambda text: lemmatize_words(text))
    return df

In [ ]:
df_imdb.isna().sum()

review       0
sentiment    0
dtype: int64

In [ ]:
df_imdb = clean(df_imdb,'review')

In [ ]:
X = df_imdb['review']
y = df_imdb['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [ ]:
## Creating bag of words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(2, 2),max_features=15000)
X_train_cv = cv.fit_transform(X_train).toarray()
X_test_cv = cv.transform(X_test).toarray()

## Creating tf-idf model
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features=15000)
X_train_tv = tv.fit_transform(X_train).toarray()
X_test_tv = tv.transform(X_test).toarray()

In [ ]:
X_test_cv.shape,X_test_tv.shape

((10000, 15000), (10000, 15000))

In [ ]:
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [ ]:
def train_models(X_train, X_test, y_train, y_test):
    """
    Trains multiple models and prints the performance metrics in a tabular format.
    """
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC

    # Splitting the dataset


    # Define models
    models = {
        # "Multinomial NB": MultinomialNB(),
        # "SVM": SVC(kernel='linear', probability=True),
        # "KNeighbors": KNeighborsClassifier(n_jobs=-1),
        "LogisticRegression": LogisticRegression(n_jobs=-1),
        "Decision Tree": DecisionTreeClassifier(),
        "Random Forest": RandomForestClassifier(n_jobs=-1)
    }

    # Initialize list to store results
    results = []

    # Train and evaluate models
    for name, model in tqdm(models.items()):
        print(f"\n🔹 Training {name}...")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred) * 100
        precision = precision_score(y_test, y_pred) * 100
        recall = recall_score(y_test, y_pred) * 100
        f1 = f1_score(y_test, y_pred) * 100

        # Store results
        results.append([name, round(accuracy, 2), round(precision, 2), round(recall, 2), round(f1, 2)])

    # Convert results to DataFrame
    df = pd.DataFrame(results, columns=["Classifier", "A", "P", "R", "F1"])

    # # Print in tabular format
    # print("\n📊 **IMDB Dataset Performance**\n")
    # print(tabulate(df, headers="keys", tablefmt="grid"))

    return df  # Returning DataFrame for further use

In [ ]:
train_models(X_train_cv, X_test_cv, y_train, y_test)

  0%|                                                                                    | 0/3 [00:00<?, ?it/s]


🔹 Training LogisticRegression...


 33%|█████████████████████████                                                  | 1/3 [04:09<08:19, 249.97s/it]


🔹 Training Decision Tree...


 67%|████████████████████████████████████████████████                        | 2/3 [1:02:50<36:14, 2174.12s/it]


🔹 Training Random Forest...


100%|████████████████████████████████████████████████████████████████████████| 3/3 [1:07:46<00:00, 1355.47s/it]


,Classifier,A,P,R,F1
0,LogisticRegression,83.14,82.97,83.40,83.18
1,Decision Tree,71.96,71.89,72.12,72.00
2,Random Forest,79.09,79.10,79.08,79.09


In [ ]:
train_models(X_train_tv, X_test_tv, y_train, y_test)

  0%|                                                                                    | 0/3 [00:00<?, ?it/s]


🔹 Training LogisticRegression...


 33%|█████████████████████████                                                  | 1/3 [02:04<04:08, 124.24s/it]


🔹 Training Decision Tree...


 67%|██████████████████████████████████████████████████                         | 2/3 [16:39<09:25, 565.76s/it]


🔹 Training Random Forest...


100%|███████████████████████████████████████████████████████████████████████████| 3/3 [18:05<00:00, 361.70s/it]


,Classifier,A,P,R,F1
0,LogisticRegression,89.41,88.27,90.90,89.57
1,Decision Tree,72.11,72.39,71.48,71.93
2,Random Forest,85.31,85.80,84.62,85.21


In [ ]:
# display memory usage in gigabytes (GB) free -h
!free -g

              total        used        free      shared  buff/cache   available
Mem:             94          18          67           0           8          75
Swap:             7           1           6


In [ ]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           94Gi        18Gi        67Gi       2.0Mi       8.4Gi        75Gi
Swap:         8.0Gi       1.8Gi       6.2Gi
